# MTG Analysis

In [35]:
# Imports
import pandas as pd
import json
import re
from pathlib import Path
import string

### Functions

In [36]:
# Iterate through formatted JSON string provided and collect only the desired information from each card.
# ID, name, color, mana cost, creature types (optional), flavor text
#def stripJSON(json_string):
    # Split on the string }\r\n],\r\n" and append it to each string slice.
    #slices = re.split(r"}\n],\n\"", json_string)
    #print("Number of slices: " + str(slices.__len__()))
    #json_string.split('''}\r\n],\r\n"''')
    #for slice in slices:
     #   slice += '''}\r\n],\r\n"'''
      #  print(slice)
       # break

### Data Cleaning

In [37]:
# Import data from JSON files and remove all irrelevant data.

# I only want: multiverse ID, color identity, name, creature types, flavor text, first printing
# Drop: all non-English text, all data relating to art, etc.

In [42]:
# Load the JSON data
json_path = Path("./data/StandardAtomic.json")
data = json.load(open(json_path))
cards = data['data']

# Flattening the data and extracting relevant information
cards_data = []
for card_name, card_list in cards.items():
    for card in card_list:
        scryfallOracleId = card["identifiers"].get("scryfallOracleId", None)
        color_identity = ','.join(card.get("colorIdentity", []))
        name = card["name"]
        text = card.get("text", "")
        first_printing = card.get("firstPrinting", "")
        creature_types = ','.join(card.get("subtypes", []))
        card_type = card.get("type", "")
        
        cards_data.append({
            "scryfall_oracle_id": scryfallOracleId,
            "color_identity": color_identity,
            "name": name,
            "creature_types": creature_types,
            "text": text,
            "first_printing": first_printing,
            "card_type": card_type
        })

# Creating DataFrame
df_cards = pd.DataFrame(cards_data)

In [39]:
# Display the DataFrame
df_cards

,scryfall_oracle_id,color_identity,name,creature_types,text,first_printing,card_type
0,79d577a8-53c0-4220-bc0e-70764d72d5fd,G,A Killer Among Us,,"When A Killer Among Us enters the battlefield,...",MKM,Enchantment
1,1167fe30-32b8-4381-8067-a05cd345977b,U,A Little Chat,,"Casualty 1 (As you cast this spell, you may sa...",SNC,Instant
2,682a0770-8fe0-4142-aa0c-b60af3628a33,W,A Tale for the Ages,,Enchanted creatures you control get +2/+2.,WOE,Enchantment
3,5404c6a5-399d-4237-b03c-41b6a1b79050,R,Abandon the Post,,Up to two target creatures can't block this tu...,MID,Sorcery
4,f9db72dc-9a5b-48a4-a86e-7464d9a2166a,R,Abrade,,Choose one —\n• Abrade deals 3 damage to targe...,HOU,Instant
...,...,...,...,...,...,...,...
3614,b168e4e6-f572-4d9f-b98f-95b2611354cb,G,"Zopandrel, Hunger Dominus","Phyrexian,Horror","Reach\nAt the beginning of each combat, double...",ONE,Legendary Creature — Phyrexian Horror
3615,e90f298f-c54b-4fe8-82db-5b9153a57712,"B,R",Zoyowa Lava-Tongue,"Goblin,Warlock","Deathtouch\nAt the beginning of your end step,...",LCI,Legendary Creature — Goblin Warlock
3616,17e11f0c-4ac7-4060-9e0b-27cd5c151e50,R,Zoyowa's Justice,,The owner of target artifact or creature with ...,LCI,Instant
3617,cfc5dc9c-6f26-460b-a624-3681b8584eb3,"B,U,W","Zur, Eternal Schemer","Human,Wizard",Flying\nEnchantment creatures you control have...,DMU,Legendary Creature — Human Wizard


In [44]:
# Take only desired columns from dataframe
df_cards_trimmed = df_cards.drop(["text", "first_printing", "card_type"], axis=1)
df_cards_trimmed

,scryfall_oracle_id,color_identity,name,creature_types
0,79d577a8-53c0-4220-bc0e-70764d72d5fd,G,A Killer Among Us,
1,1167fe30-32b8-4381-8067-a05cd345977b,U,A Little Chat,
2,682a0770-8fe0-4142-aa0c-b60af3628a33,W,A Tale for the Ages,
3,5404c6a5-399d-4237-b03c-41b6a1b79050,R,Abandon the Post,
4,f9db72dc-9a5b-48a4-a86e-7464d9a2166a,R,Abrade,
...,...,...,...,...
3614,b168e4e6-f572-4d9f-b98f-95b2611354cb,G,"Zopandrel, Hunger Dominus","Phyrexian,Horror"
3615,e90f298f-c54b-4fe8-82db-5b9153a57712,"B,R",Zoyowa Lava-Tongue,"Goblin,Warlock"
3616,17e11f0c-4ac7-4060-9e0b-27cd5c151e50,R,Zoyowa's Justice,
3617,cfc5dc9c-6f26-460b-a624-3681b8584eb3,"B,U,W","Zur, Eternal Schemer","Human,Wizard"
